# Flux estimates for each aircraft campaign

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import emergent_constraint as ec

In [3]:
clobber = False
clobber_deep = False
air_parms = ec.get_parameters('default')

obj_air = {};
acs = {}; scs = {};
for constraint_type in ['ocean_constraint',]:
    model_input_lists = ec.get_model_tracer_lists(constraint_type)
    obj_air[constraint_type] = ec.whole_enchilada(clobber=clobber_deep, **model_input_lists)    
    acs[constraint_type] = obj_air[constraint_type].get_ac(**air_parms, clobber=clobber)

loading pfp
loading medusa
loading obs
computing...
assuming cache is correct
reading cached file: /glade/u/home/mclong/p/sh-co2-hole/models/../../sh-co2-hole-data/models/data/CT2017/flux_ts_monthly.lat_range=-90_0_to_-45_0.zarr
assuming cache is correct
reading cached file: /glade/u/home/mclong/p/sh-co2-hole/models/../../sh-co2-hole-data/models/data/CT2019B/flux_ts_monthly.lat_range=-90_0_to_-45_0.zarr
assuming cache is correct
reading cached file: /glade/u/home/mclong/p/sh-co2-hole/models/../../sh-co2-hole-data/models/data/CTE2018/flux_ts_monthly.lat_range=-90_0_to_-45_0.zarr
assuming cache is correct
reading cached file: /glade/u/home/mclong/p/sh-co2-hole/models/../../sh-co2-hole-data/models/data/CTE2020/flux_ts_monthly.lat_range=-90_0_to_-45_0.zarr
assuming cache is correct
reading cached file: /glade/u/home/mclong/p/sh-co2-hole/models/../../sh-co2-hole-data/models/data/MIROC/flux_ts_monthly.lat_range=-90_0_to_-45_0.zarr
assuming cache is correct
reading cached file: /glade/u/home/

In [4]:
ac = acs['ocean_constraint']

In [5]:
dfo = ac.gradient_summary[['vg_obs', 'vg_obs_std', 'vg_ext', 'vg_ext_std',]]
dfo = dfo.rename(
    columns=dict(
        vg_obs='Observed Vertical Gradient [ppm]',
        vg_obs_std='Observed Gradient Uncertainty [ppm]',
        vg_ext='Simulated Land+Fossil Gradient [ppm]',
        vg_ext_std='Simulated Land+Fossil Gradient [ppm]',        
    )
)

# add columns for fluxes, flux errors, and start/end of 90-day windows?
dfo['Flux Date Start'] = [str(d[0]) for d in ac.campaign_flux.loc[dfo.index].flux_window_date_bound]
dfo['Flux Date End'] = [str(d[1]) for d in ac.campaign_flux.loc[dfo.index].flux_window_date_bound]
dfo['Flux Estimate [Pg C/yr]'] = ac.campaign_flux.loc[dfo.index].flux
dfo['Flux Uncertainty [Pg C/yr]'] = ac.campaign_flux.loc[dfo.index].flux_error

dfo

,Observed Vertical Gradient [ppm],Observed Gradient Uncertainty [ppm],Simulated Land+Fossil Gradient [ppm],Simulated Land+Fossil Gradient [ppm],Flux Date Start,Flux Date End,Flux Estimate [Pg C/yr],Flux Uncertainty [Pg C/yr]
campaign,,,,,,,,
HIPPO-1,-1.071490,0.460398,-0.023900,0.120961,2008-10-26,2009-01-24,-1.070784,0.642003
HIPPO-2,-0.189390,0.091089,-0.046578,0.084528,2009-08-17,2009-11-15,-0.240134,0.472764
HIPPO-3,-0.760160,0.261751,-0.173505,0.026798,2010-01-09,2010-04-09,-1.311428,1.075345
HIPPO-5,-0.308040,0.096509,-0.306848,0.060584,2011-06-04,2011-09-02,0.101691,0.497746
ORCAS-J,-1.180725,0.074389,-0.070150,0.112105,2015-11-01,2016-01-30,-1.131714,0.395743
ORCAS-F,-1.475288,0.077335,-0.075252,0.129962,2015-12-01,2016-02-29,-1.411733,0.556450
ATOM-1,-0.282512,0.116986,-0.208775,0.058863,2016-05-17,2016-08-15,-0.073410,0.497385
ATOM-2,-1.672800,0.157935,0.010962,0.081646,2016-11-15,2017-02-13,-1.686205,0.727059
ATOM-3,-0.187130,0.117521,-0.152210,0.110920,2017-07-19,2017-10-17,0.020283,0.579938
